# AutoGen Examples: Building Conversational AI Agent Systems

This notebook provides hands-on examples of using Microsoft's AutoGen framework for building conversational AI agent systems. You can run these examples to gain practical experience with this innovative framework.

## Helpful Resources

- [Official AutoGen Documentation](https://microsoft.github.io/autogen/)
- [AutoGen GitHub Repository](https://github.com/microsoft/autogen)
- [AutoGen Examples Gallery](https://microsoft.github.io/autogen/docs/Examples)
- [AutoGen Discord Community](https://discord.gg/pAbnFJrkgZ)
- [AutoGen API Reference](https://microsoft.github.io/autogen/docs/reference/)
- [AutoGen Research Paper](https://arxiv.org/abs/2308.08155)

## Setup Instructions

First, let's install the necessary packages:

In [ ]:
# Install required packages
!pip install pyautogen

# For comprehensive installation with all features
# !pip install "pyautogen[all]"

# For Azure OpenAI support (already included in base package)
!pip install openai

## Setting API Keys and Configurations

For these examples to work, you'll need to set your API keys for OpenAI or Azure OpenAI. Replace the placeholders with your actual API keys.

In [ ]:
import os
import autogen

# Option 1: OpenAI Setup
# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual API key

# OpenAI configuration
openai_config_list = [
    {
        'model': 'gpt-4',
        'api_key': os.environ["OPENAI_API_KEY"]
    }
]

# Option 2: Azure OpenAI Setup
# Set your Azure OpenAI variables
# os.environ["AZURE_OPENAI_API_KEY"] = "your-azure-openai-api-key"  # Replace with your actual Azure API key
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource-name.openai.azure.com/"  # Replace with your endpoint

# Azure OpenAI configuration
azure_openai_config_list = [
    {
        'model': 'gpt-4',  # The deployment name you chose when you deployed the GPT-4 model
        'api_key': os.environ.get("AZURE_OPENAI_API_KEY", ""),
        'api_type': 'azure',
        'api_base': os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
        'api_version': '2023-05-15'
    }
]

# Select which configuration to use (OpenAI or Azure OpenAI)
config_list = openai_config_list  # Change to azure_openai_config_list to use Azure

# Alternatively, you can load from a configuration file
# config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

## Example 1: Basic Two-Agent Conversation

Let's start with a simple conversation between two agents: an AI assistant and a user proxy.

## Using Azure OpenAI with AutoGen

Here's how to explicitly configure and use Azure OpenAI with AutoGen:

In [ ]:
# Azure OpenAI Configuration
from typing import Literal

# Define Azure OpenAI configuration (uncomment and fill in your actual values)
# os.environ["AZURE_OPENAI_API_KEY"] = "your-azure-openai-api-key"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource-name.openai.azure.com/"

# Configure for Azure OpenAI
azure_config_list = [
    {
        "model": "gpt-4",  # This should be the deployment name you chose
        "api_type": "azure",
        "api_version": "2023-05-15",
        "api_key": os.environ.get("AZURE_OPENAI_API_KEY", ""),
        "api_base": os.environ.get("AZURE_OPENAI_ENDPOINT", "")
    }
]

# Azure OpenAI-specific settings
azure_llm_config = {
    "config_list": azure_config_list,
    "temperature": 0
}

# Create agents with Azure OpenAI
assistant_with_azure = autogen.AssistantAgent(
    name="Azure_Assistant",
    llm_config=azure_llm_config,
    system_message="You are a helpful AI assistant powered by Azure OpenAI. Respond concisely."
)

user_proxy_with_azure = autogen.UserProxyAgent(
    name="User_With_Azure",
    human_input_mode="NEVER",  # Don't ask for human input in this example
    max_consecutive_auto_reply=5,
    code_execution_config={"work_dir": "coding_azure", "use_docker": False},
    system_message="You are a proxy for the user testing Azure OpenAI integration."
)

# To run this conversation (uncomment when you've configured Azure OpenAI)
'''
user_proxy_with_azure.initiate_chat(
    assistant_with_azure,
    message="Can you write a Python function that generates a Fibonacci sequence up to n terms?"
)
'''

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# Create an assistant agent
assistant = AssistantAgent(
    name="AI_Assistant",
    llm_config={"config_list": config_list},
    system_message="You are a helpful AI assistant. Respond concisely and clearly."
)

# Create a user proxy agent that doesn't require human input
user_proxy = UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",  # Don't ask for human input in this example
    max_consecutive_auto_reply=5,  # Limit the conversation length
    system_message="You are a proxy for the user. Ask the AI assistant questions about AI frameworks."
)

# Start the conversation
user_proxy.initiate_chat(
    assistant,
    message="What are the key differences between LangChain and AutoGen frameworks?"
)

## Example 2: Agent with Code Execution

One of AutoGen's strengths is its ability to generate and execute code. Let's create an agent that can write and run Python code.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent
import os

# Ensure we have a coding workspace
os.makedirs("coding_workspace", exist_ok=True)

# Create an assistant agent focused on coding
coding_assistant = AssistantAgent(
    name="Coding_Assistant",
    llm_config={"config_list": config_list},
    system_message="You are a Python expert who writes efficient, well-documented code. When asked to code, provide complete and executable solutions."
)

# Create a user proxy agent with code execution capabilities
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="TERMINATE",  # Only ask for human input when termination is requested
    code_execution_config={
        "work_dir": "coding_workspace",
        "use_docker": False  # Set to True to use Docker for isolation
    }
)

# Start the conversation with a coding task
user_proxy.initiate_chat(
    coding_assistant,
    message="Write a Python script that creates a web server to display 'Hello, World!' when accessed."
)

## Example 3: Multi-Agent Group Chat

AutoGen excels at facilitating conversations between multiple specialized agents. Let's create a group chat with several agents that collaborate on a task.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# Create specialized agents for different roles
programmer = AssistantAgent(
    name="Programmer",
    llm_config={"config_list": config_list},
    system_message="You are an expert programmer specializing in Python. You write clean, efficient, and well-documented code."
)

data_scientist = AssistantAgent(
    name="Data_Scientist",
    llm_config={"config_list": config_list},
    system_message="You are a data scientist who excels at data analysis, visualization, and machine learning."
)

product_manager = AssistantAgent(
    name="Product_Manager",
    llm_config={"config_list": config_list},
    system_message="You are a product manager who focuses on user needs and ensuring that technical solutions solve real problems effectively."
)

# Create a user proxy agent
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="TERMINATE",  # Only ask for human input when termination is requested
    code_execution_config={"work_dir": "coding_workspace", "use_docker": False}
)

# Create a group chat with all agents
groupchat = GroupChat(
    agents=[user_proxy, programmer, data_scientist, product_manager],
    messages=[],
    max_round=12
)

# Create a manager for the group chat
manager = GroupChatManager(groupchat=groupchat)

# Start the conversation with a complex problem that requires collaboration
user_proxy.initiate_chat(
    manager,
    message="We need to build a dashboard that visualizes Twitter sentiment about AI technologies. Please work together to create a plan and some sample code."
)

## Example 4: Human-in-the-Loop Interaction

AutoGen allows for human feedback during agent conversations. Let's create an example where human input is requested at specific points.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# Create an assistant agent
assistant = AssistantAgent(
    name="AI_Assistant",
    llm_config={"config_list": config_list},
    system_message="You are an AI assistant helping a user build a data analysis project. Ask clarifying questions when needed."
)

# Create a user proxy agent that requests human input at key decision points
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",  # Always ask for human input when the agent is prompted
    code_execution_config={"work_dir": "coding_workspace", "use_docker": False}
)

# Start the conversation
user_proxy.initiate_chat(
    assistant,
    message="I want to build a data visualization project showing climate change trends over time."
)

## Example 5: Custom Function Calling

Let's implement a scenario where agents can use custom functions to perform specific tasks.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent
import json

# Define custom functions that agents can use
def search_database(query):
    """Simulated database search function"""
    # In a real scenario, this would connect to a database
    mock_database = {
        "AI frameworks": ["LangChain", "AutoGen", "CrewAI", "LangGraph"],
        "Programming languages": ["Python", "JavaScript", "Rust", "Go"],
        "Cloud providers": ["AWS", "Azure", "Google Cloud", "IBM Cloud"]
    }
    
    for category, items in mock_database.items():
        if query.lower() in category.lower():
            return json.dumps({"category": category, "items": items})
    
    return json.dumps({"error": "No results found", "query": query})

def calculate_statistics(data_string):
    """Calculate basic statistics on a list of numbers"""
    try:
        # Convert the input string to a list of numbers
        data = [float(x) for x in data_string.split(',')]
        result = {
            "count": len(data),
            "mean": sum(data) / len(data) if data else 0,
            "min": min(data) if data else None,
            "max": max(data) if data else None
        }
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"error": str(e)})

# Create the assistant agent
function_assistant = AssistantAgent(
    name="Function_Assistant",
    llm_config={"config_list": config_list},
    system_message="You are an AI assistant with access to special functions. You can search a database with search_database(query) and calculate statistics with calculate_statistics(data_string)."
)

# Create a user proxy agent with registered functions
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="TERMINATE",
    function_map={
        "search_database": search_database,
        "calculate_statistics": calculate_statistics
    }
)

# Start the conversation
user_proxy.initiate_chat(
    function_assistant,
    message="What AI frameworks are available in the database? Also, can you calculate statistics for this data: 10, 15, 20, 25, 30?"
)

## Example 6: Iterative Task Solving with ReAct

AutoGen can implement the ReAct (Reasoning + Acting) pattern for iterative task solving.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# Create an assistant agent with ReAct-like capabilities
react_assistant = AssistantAgent(
    name="ReAct_Assistant",
    llm_config={
        "config_list": config_list,
        "temperature": 0,  # Lower temperature for more deterministic reasoning
    },
    system_message="""You are an assistant that carefully solves tasks step by step.
For complex problems:
1. Break down the problem into smaller steps
2. Reason through each step individually
3. Verify your reasoning before providing the final answer
If you need to write code, make it executable and test it thoroughly.
Always show your full reasoning process."""
)

# Create a user proxy agent with code execution
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="TERMINATE",
    code_execution_config={"work_dir": "coding_workspace", "use_docker": False}
)

# Start the conversation with a complex problem
user_proxy.initiate_chat(
    react_assistant,
    message="Solve this programming challenge: Write a function that finds all prime numbers up to n using the Sieve of Eratosthenes algorithm. Then use it to count how many prime numbers are there between 1 and 1000."
)

## Further Exploration

These examples demonstrate the core capabilities of AutoGen. For more advanced usage, consider exploring:

1. **Advanced Agent Customization**: Creating agents with specialized behaviors and reasoning patterns
2. **More Complex Group Chat Dynamics**: Implementing multi-stage conversations with different agent participation
3. **Memory Management**: Techniques for maintaining long-term memory across conversations
4. **Docker Integration**: Secure code execution in isolated environments
5. **Integration with External Services**: Connecting agents to APIs and databases

Check out the [AutoGen documentation](https://microsoft.github.io/autogen/) for more examples and advanced usage scenarios.

## Complete Example with Configuration Choice

This example shows how to create a multi-agent system where you can easily switch between OpenAI and Azure OpenAI:

In [ ]:
# Configurable Multi-Agent Example

# Function to create config based on provider choice
def get_llm_config(provider="openai"):
    if provider == "azure":
        # Azure OpenAI configuration
        return {
            "config_list": [
                {
                    "model": "gpt-4",  # Your deployment name
                    "api_type": "azure",
                    "api_version": "2023-05-15",
                    "api_key": os.environ.get("AZURE_OPENAI_API_KEY", ""),
                    "api_base": os.environ.get("AZURE_OPENAI_ENDPOINT", "")
                }
            ],
            "temperature": 0
        }
    else:  # Default to OpenAI
        # OpenAI configuration
        return {
            "config_list": [
                {
                    "model": "gpt-4",
                    "api_key": os.environ.get("OPENAI_API_KEY", "")
                }
            ],
            "temperature": 0
        }

# Choose your provider here
provider = "openai"  # Change to "azure" to use Azure OpenAI

# Get appropriate config
llm_config = get_llm_config(provider)

# Create a coding agent
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
    system_message="You are a Python expert. Write efficient, well-documented code based on requirements."
)

# Create a code reviewer agent
reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config=llm_config,
    system_message="You are a code reviewer who focuses on code quality, security, and best practices."
)

# Create a user proxy agent with code execution capability
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding_workspace", "use_docker": False},
    system_message="You represent the user in this conversation."
)

# Create a group chat
from autogen.agentchat.groupchat import GroupChat
from autogen.agentchat.agent import Agent
from typing import Dict, List, Optional, Union

def select_speaker(groupchat: GroupChat, messages: List[Dict]):
    """Function to select the next speaker in the group chat."""
    # Extract last message's content and speaker
    last_message = messages[-1]
    last_speaker = last_message["sender"]
    content = last_message["content"]
    
    # Default to user_proxy for final decisions or when input is needed
    if "should we implement this?" in content.lower() or "what do you think?" in content.lower():
        return user_proxy
    
    # If the last message was from the user, default to coder
    if last_speaker == user_proxy.name:
        return coder
    
    # If the message contains code and was from coder, let reviewer speak
    if "```" in content and last_speaker == coder.name:
        return reviewer
    
    # If reviewer just gave feedback, let coder implement changes
    if last_speaker == reviewer.name:
        return coder
    
    # Default back to reviewer for another round of feedback
    return reviewer

# Set up the group chat
groupchat = GroupChat(
    agents=[user_proxy, coder, reviewer],
    messages=[],
    max_round=12,
    speaker_selection_method=select_speaker
)

# Create a manager for the group chat
manager = autogen.GroupChatManager(groupchat=groupchat)

# Start the conversation (uncomment to run)
'''
user_proxy.initiate_chat(
    manager,
    message="Create a Python script that analyzes a CSV file containing sales data and generates a bar chart of monthly revenue."
)
'''